In [1]:
import json
from pathlib import Path
import os
from os import getenv

from tqdm import tqdm
import mlflow
import openai
import pandas as pd
# from sentencepiece import SentencePieceProcessor
from tinygrad import Device, Tensor, nn, Variable, GlobalCounters
from tinygrad.helpers import Context, Timing, Profiling, DEBUG, JIT, getenv, colored
from tinygrad.nn.state import get_state_dict, safe_load, torch_load, load_state_dict, get_parameters, safe_save
from transformers import AutoTokenizer, AutoConfig, DataCollatorWithPadding, pipeline, Trainer, TrainingArguments

from notebooks.Research.lib.transformers.src.transformers.models.llama.modeling_tinygrad_llama import TinygradLlamaForCausalLM
# from notebooks.Research.tinygrad.train_llama import function_calling_template
from timestep.config import settings

device = Device.DEFAULT
print(f'Using Tinygrad and {device} device.')

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Using Tinygrad and CUDA device.


In [2]:
# bos_token="<|begin_of_text|>"
bos_token="<s>"
bos_token_id=1
# eos_token="<|eot_id|>"
eos_token="</s>"
eos_token_id=2
pad_token=eos_token
pad_token_id=eos_token_id
# unk_token="<|unk_id|>"
unk_token="<unk>"
unk_token_id=0

In [3]:
hf_repo_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [4]:
config = AutoConfig.from_pretrained(hf_repo_id, bos_token_id=bos_token_id, eos_token_id=eos_token_id)
config

LlamaConfig {
  "_name_or_path": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [5]:
# model_path = os.path.join(settings.app_dir, f"models/{hf_repo_id}/model.safetensors")
model_path: Path = Path(settings.app_dir) / f"models/{hf_repo_id}/model.safetensors"
# tokenizer_path = os.path.join(settings.app_dir, f"models/{hf_repo_id}/tokenizer.model")

# model = TinygradLlamaForCausalLM.from_pretrained(hf_repo_id)
model = TinygradLlamaForCausalLM(
    config=config,
    device=device,
    model_path=model_path,
)

ram used:  2.20 GB, freqs_cis                                         : 100%|█| 


loaded weights in 2971.83 ms, 2.20 GB loaded at 0.74 GB/s


In [6]:
print('bos_token_id:', model.config.bos_token_id)
print('eos_token_id:', model.config.eos_token_id)
print('pad_token_id:', model.config.pad_token_id)
# print('unk_token_id:', model.config.unk_token_id)

bos_token_id: 1
eos_token_id: 2
pad_token_id: None


In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    hf_repo_id,
    # bos_token="<|begin_of_text|>",
    bos_token=bos_token,
    clean_up_tokenization_spaces=True,
    eos_token=eos_token,
    # pad_token='<|im_end|>',
    # pad_token=pad_token,
)

tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id

print('bos_token:', tokenizer.bos_token)
print('eos_token:', tokenizer.eos_token)
print('pad_token:', tokenizer.pad_token)
print('unk_token:', tokenizer.unk_token)

print('bos_token_id:', tokenizer.bos_token_id)
print('eos_token_id:', tokenizer.eos_token_id)
print('pad_token_id:', tokenizer.pad_token_id)
print('unk_token_id:', tokenizer.unk_token_id)

bos_token: <s>
eos_token: </s>
pad_token: </s>
unk_token: <unk>
bos_token_id: 1
eos_token_id: 2
pad_token_id: 2
unk_token_id: 0


In [8]:
assert tokenizer.bos_token_id == model.config.bos_token_id, f'{tokenizer.bos_token_id} != {model.config.bos_token_id}'
assert tokenizer.eos_token_id == model.config.eos_token_id, f'{tokenizer.eos_token_id} != {model.config.eos_token_id}'
# assert tokenizer.pad_token_id == model.config.pad_token_id, f'{tokenizer.pad_token_id} != {model.config.pad_token_id}'
# assert tokenizer.unk_token_id == model.config.unk_token_id, f'{tokenizer.unk_token_id} != {model.config.unk_token_id}'

In [9]:
# import tiktoken 

# cl100k_base = tiktoken.get_encoding("cl100k_base") 

# enc = tiktoken.Encoding( 
#     name="gpt-35-turbo",  
#     pat_str=cl100k_base._pat_str, 
#     mergeable_ranks=cl100k_base._mergeable_ranks, 
#     special_tokens={ 
#         **cl100k_base._special_tokens, 
#         "<|im_start|>": 100264, 
#         "<|im_end|>": 100265
#     } 
# ) 

# tokens = enc.encode( 
#     "<|im_start|>user\nHello<|im_end|><|im_start|>assistant",  
#     allowed_special={"<|im_start|>", "<|im_end|>"} 
# ) 

# assert len(tokens) == 7 
# assert tokens == [100264, 882, 198, 9906, 100265, 100264, 78191]

In [10]:
# Copied and modified from https://github.com/abetlen/llama-cpp-python/blob/658b244c5aa924fc6f4d04f92445dd8f724b6017/llama_cpp/llama_chat_format.py#L3345
# See also https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/chat-markup-language
function_calling_template = (
  "{%- for message in messages %}"
  "<|im_start|>{{- message.role }}\n"
  # System message
  "{% if message.role == 'system' %}"
  "{{ message.content }}"
  "{% if tool_calls %}"
  "\n\nYou have access to the following functions:\n"
  "{% for tool in tools %}"
  "\nfunctions.{{ tool.function.name }}:\n"
  "{{ tool.function.parameters | tojson }}"
  "\n{% endfor %}"
  "\n\nYou can respond to user messages with either a single message or one or more function calls."
  "\n\nTo respond with a message begin the message with \"message:\", use the following format:"
  "\n\nmessage:"
  "\n<message>"
  "\n\nTo respond with one or more function calls begin the message with 'functions.<function_name>:', use the following format:"
  "\n\nfunctions.<function_name>:"
  '\n{ "arg1": "value1", "arg2": "value2" }'
  "\nfunctions.<function_name>:"
  '\n{ "arg1": "value1", "arg2": "value2" }'
  "\nWhen responding with function calls, only output the function calls, do not include any additional text."
  "{% endif %}"
  "<|im_end|>\n"
  "{% endif %}"
  # User message
  "{% if message.role == 'user' %}"
  "{{ message.content }}"
  "<|im_end|>\n"
  "{% endif %}"
  # Assistant message
  "{% if message.role == 'assistant' %}"
  ## Reglar message
  "{% if message.content and message.content | length > 0 %}"
  "{% if tool_calls %}"
  "message:\n"
  "{% endif %}"
  "{{ message.content }}"
  "<|im_end|>\n"
  "{% endif %}"
  ## Function calls
  "{% if 'tool_calls' in message %}"
  "{% for tool_call in message.tool_calls %}"
  "functions.{{ tool_call.function.name }}:\n"
  "{{ tool_call.function.arguments }}"
  "{% endfor %}"
  "<|im_end|>\n"
  "{% endif %}"
  "{% endif %}"
  "{% endfor %}"
  "{% if add_generation_prompt %}<|im_start|>assistant\n{% endif %}"
)

# function_calling_template = function_calling_template.replace("<|im_start|>", tokenizer.bos_token)
function_calling_template = function_calling_template.replace("<|im_end|>", tokenizer.eos_token)

print(function_calling_template)

{%- for message in messages %}<|im_start|>{{- message.role }}
{% if message.role == 'system' %}{{ message.content }}{% if tool_calls %}

You have access to the following functions:
{% for tool in tools %}
functions.{{ tool.function.name }}:
{{ tool.function.parameters | tojson }}
{% endfor %}

You can respond to user messages with either a single message or one or more function calls.

To respond with a message begin the message with "message:", use the following format:

message:
<message>

To respond with one or more function calls begin the message with 'functions.<function_name>:', use the following format:

functions.<function_name>:
{ "arg1": "value1", "arg2": "value2" }
functions.<function_name>:
{ "arg1": "value1", "arg2": "value2" }
When responding with function calls, only output the function calls, do not include any additional text.{% endif %}</s>
{% endif %}{% if message.role == 'user' %}{{ message.content }}</s>
{% endif %}{% if message.role == 'assistant' %}{% if message

In [11]:
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

# llama_3_1_8b_instruct_chat_template = "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n{%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim + '<|eot_id|>' }}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n                {%- endfor %}\n            {{- \")\" }}\n        {%- else  %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we're in ipython mode #}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}\n"

# llama_3_1_8b_instruct_chat_template = "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n{%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim }}\n        {%- if not loop.last or add_generation_prompt %}\n            {{- '<|eot_id|>' }}\n        {%- endif %}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n                {%- endfor %}\n            {{- \")\" }}\n        {%- else  %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we're in ipython mode #}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}\n"

llama_3_1_8b_instruct_chat_template = "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n{%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim }}\n        {%- if not loop.last or add_generation_prompt %}\n            {{- '<|eot_id|>' }}\n        {%- endif %}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n                {%- endfor %}\n            {{- \")\" }}\n        {%- else  %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we're in ipython mode #}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}\n"

llama_3_1_8b_instruct_chat_template = llama_3_1_8b_instruct_chat_template.replace("<|eot_id|>", f"{tokenizer.eos_token}\n")

print(llama_3_1_8b_instruct_chat_template)

{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "\n\n"}}
{%- endif

In [12]:
# template = """{%- for message in messages %}
# {%- if message['role'] == 'user' %}
# {{- '<|user|>
# ' + message['content'] + eos_token + '\n' }}
# {%- elif message['role'] == 'system' %}
# {{- '<|system|>
# ' + message['content'] + 
# eos_token + '\n' }}
# {%- elif message['role'] == 'assistant' %}
# {{- '<|assistant|>
# '  + message['content'] + 
# eos_token + '\n' }}
# {%- endif %}
# {%- if loop.last and add_generation_prompt %}
# {{- '<|assistant|>' }}
# {%- endif %}
# {%- endfor %}"""

template = """{%- for message in messages %}
{%- if message['role'] == 'assistant' %}
{{- '<|assistant|>' + '\n' }}
{%- if message['content'] %}
{{- message['content'] + eos_token + '\n' }}
{%- elif message['tool_calls'] %}
{{- 'tool_calls:' + '\n' }}
{%- for tool_call in message['tool_calls'] %}
{{- tool_call['function']['name'] + ': ' + tool_call['function']['arguments'] }}
{%- endfor %}
{{- eos_token + '\n' }}
{%- endif %}
{%- elif message['role'] == 'system' %}
{{- '<|system|>' + '\n' }}
{{- message['content'] }}
{%- if tools %}
{{- '\n\nYou are aware of the following tools:\n' }}
{%- for tool in tools %}
{{- tool.function.name }}: {{ tool.function.parameters | tojson }}
{%- endfor %}
{{- '\nYou can suggest tool calls by responding in the following format:\n' }}
{{- 'tool_calls:' + '\n' }}
{{- 'tool_name: {\"arg1\": \"value1\", \"arg2\": \"value2\"}' }}
{{- eos_token + '\n' }}
{%- endif %}
{%- elif message['role'] == 'user' %}
{{- '<|user|>' + '\n' }}
{{- message['content'] + eos_token + '\n' }}
{%- endif %}
{%- if loop.last and add_generation_prompt %}
{{- '<|assistant|>' + '\n' }}
{%- endif %}
{%- endfor %}"""

In [13]:
# open("template.jinja", "w").write(template)
# open("template.jinja", "w").write(tokenizer.chat_template)
open("template.jinja", "w").write(function_calling_template)
# open("template.jinja", "w").write(llama_3_1_8b_instruct_chat_template)

tokenizer.chat_template = open("template.jinja").read()

In [14]:
conversations = []

with open("../../data/drone_training.jsonl") as f:
    for line in f:
        conversations.append(json.loads(line))

conversation = conversations[0]

# print('messages: ', conversation["messages"])

# print('parallel_tool_calls: ', conversation["parallel_tool_calls"])

# print('tools: ', conversation["tools"])

# conversation = conversation["messages"][0:2]
# conversation = conversation["messages"]
conversation["messages"]

[{'role': 'system',
  'content': 'You are an intelligent AI that controls a drone. Given a command or request from the user,\ncall one of your functions to complete the request. If the request cannot be completed by your available functions, call the reject_request function.\nIf the request is ambiguous or unclear, reject the request.'},
 {'role': 'user',
  'content': "Let's get the drone in the air, how high should it go?"},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_id',
    'type': 'function',
    'function': {'name': 'takeoff_drone', 'arguments': '{"altitude": 100}'}}]}]

In [15]:
from datetime import datetime


def generate_prompt(add_generation_prompt, date_string=f"{datetime.now():%d %b %Y}", messages=[], tool_calls=True, tools=[]):
    return tokenizer.apply_chat_template(
        add_generation_prompt=add_generation_prompt,
        # add_special_tokens=False,
        # conversation=conversation,
        # conversation=conversation["messages"][0:2], # Skip tool messages for now
        # conversation=conversation["messages"],
        conversation=messages,
        date_string=date_string,
        # return_dict=True,
        return_tensors=False,
        tokenize=False,
        tool_calls=tool_calls,
        # tools=conversation["tools"],
        tools=tools,
    )

prompt = generate_prompt(
    add_generation_prompt=True,
    messages=conversation["messages"][0:2],
    # messages=conversation["messages"],
    tool_calls=True,
    tools=conversation["tools"],
)

print(prompt)

<|im_start|>system
You are an intelligent AI that controls a drone. Given a command or request from the user,
call one of your functions to complete the request. If the request cannot be completed by your available functions, call the reject_request function.
If the request is ambiguous or unclear, reject the request.
You have access to the following functions:
functions.takeoff_drone:
{"type": "object", "properties": {"altitude": {"type": "integer"}}, "required": ["altitude"]}
functions.land_drone:
{"type": "object", "properties": {"location": {"type": "string", "enum": ["current", "home_base", "custom"]}, "coordinates": {"type": "object"}}, "required": ["location"]}
functions.control_drone_movement:
{"type": "object", "properties": {"direction": {"type": "string", "enum": ["forward", "backward", "left", "right", "up", "down"]}, "distance": {"type": "integer"}}, "required": ["direction", "distance"]}
functions.set_drone_speed:
{"type": "object", "properties": {"speed": {"type": "integ

In [16]:
hf_tokenized_prompt = tokenizer.tokenize(prompt, add_special_tokens=False)
hf_encoded_tokenized_prompt = tokenizer.convert_tokens_to_ids(hf_tokenized_prompt)
hf_encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False) # Same as doing self.convert_tokens_to_ids(self.tokenize(text))

assert hf_encoded_prompt == hf_encoded_tokenized_prompt, f"\n{hf_encoded_prompt}\n!=\n{hf_encoded_tokenized_prompt}"

# decoded_hf_encoded_prompt = tokenizer.decode(hf_encoded_prompt, skip_special_tokens=True, clean_up_tokenization_spaces=False)
# decoded_hf_encoded_prompt = tokenizer.decode(hf_encoded_prompt, skip_special_tokens=True)
decoded_hf_encoded_prompt = tokenizer.decode(hf_encoded_prompt)

# assert decoded_hf_encoded_prompt == prompt, f"\n{decoded_hf_encoded_prompt}\n!=\n{prompt}"

for line_a, line_b in zip(prompt.split("\n"), decoded_hf_encoded_prompt.split("\n")):
    try:
        assert line_b == line_a

    except AssertionError:
        print(f"{line_b}\n!=\n{line_a}")
        assert line_b.strip() == line_a

When responding with function calls, only output the function calls, do not include any additional text.</s> 
!=
When responding with function calls, only output the function calls, do not include any additional text.</s>
Let's get the drone in the air, how high should it go?</s> 
!=
Let's get the drone in the air, how high should it go?</s>


In [17]:
def predict(prompt: str):
    inputs = tokenizer(prompt, return_tensors="np", add_special_tokens=False)
    inputs = {k: Tensor(v, device=device) for k, v in inputs.items()}

    outputs = model.generate(**inputs, do_sample=True, max_new_tokens=256, temperature=0.0)

    # decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
    decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):])

    return decoded_output

In [18]:
expected_output = generate_prompt(
    add_generation_prompt=False,
    messages=conversation["messages"][2:],
    tool_calls=True,
    tools=conversation["tools"],
)

expected_output = "\n".join(expected_output.split("\n")[1:]) # Skip the first line

print('expected_output:')
print(expected_output)

decoded_output = predict(prompt)

print('decoded_output:')
print(decoded_output)

expected_output:
functions.takeoff_drone:
{"altitude": 100}</s>

decoded_output:
Sure! To set the drone's altitude, you can use the "takeoff_drone" function. Here's an example:

```
functions.takeoff_drone(altitude=500)
```

This will take the drone up to an altitude of 500 meters (1640 feet). You can also pass in additional parameters to customize the takeoff, such as the drone's current location, speed, and direction.</s>


In [19]:
from difflib import SequenceMatcher

def compute_similarity_metric(prediction: str, target: str):
    s_1 = SequenceMatcher(None, prediction, target)
    s_2 = SequenceMatcher(None, target, prediction)

    return (s_1.ratio() + s_2.ratio()) / 2

similarity_metric = compute_similarity_metric(decoded_output, expected_output)
print('similarity_metric:', similarity_metric)

similarity_metric: 0.14249363867684478


In [20]:
input_prompts = [
    generate_prompt(
        # add_generation_prompt=False,
        add_generation_prompt=True,
        messages=conversation["messages"][0:2], # TODO: do this until we reach the first assistant message where where "weight" is undefined or 1
        tools=conversation["tools"],
    )
    for conversation in conversations
]

ground_truth_responses = [
    generate_prompt(
        add_generation_prompt=False,
        messages=conversation["messages"][2:], # TODO: Start at the fisrt assistant message where "weight" is undefined or 1
        tools=conversation["tools"],
    )
    for conversation in conversations
]

# expected_output = "\n".join(expected_output.split("\n")[1:]) # Skip the first line

for i in range(len(ground_truth_responses)):
    ground_truth_responses[i] = "\n".join(ground_truth_responses[i].split("\n")[1:]) # Skip the first line

len(input_prompts), len(ground_truth_responses)

(103, 103)

In [21]:
from tqdm.notebook import trange

stop_after = len(input_prompts)

eval_df = pd.DataFrame({
    "inputs": input_prompts[:stop_after],
    "ground_truth": ground_truth_responses[:stop_after],
})
predictions = []

# for i, row in tqdm(eval_df.iterrows()):
for i in trange(len(eval_df)):
    # print('i: ', i)
    row = eval_df.iloc[i]

    if i >= stop_after:
        break

    input_prompt = row["inputs"]
    ground_truth_response = row["ground_truth"]

    # print('input_prompt: ')
    # print(input_prompt)
    # print('ground_truth_response: ')
    # print(ground_truth_response)

    prediction = predict(input_prompt)
    predictions.append(prediction)

    # print('prediction: ')
    # print(prediction)

eval_df["predictions"] = predictions

eval_df["similarity_metric"] = eval_df.apply(lambda row: compute_similarity_metric(row["predictions"], row["ground_truth"]), axis=1)

  0%|          | 0/103 [00:00<?, ?it/s]

In [22]:
print('prompt:')
print(eval_df["inputs"].values[0])

print('target:')
print(eval_df["ground_truth"].values[0])

print('prediction:')
print(eval_df["predictions"].values[0])

print('similarity_metric:')
print(eval_df["similarity_metric"].values[0])

prompt:
<|im_start|>system
You are an intelligent AI that controls a drone. Given a command or request from the user,
call one of your functions to complete the request. If the request cannot be completed by your available functions, call the reject_request function.
If the request is ambiguous or unclear, reject the request.
You have access to the following functions:
functions.takeoff_drone:
{"type": "object", "properties": {"altitude": {"type": "integer"}}, "required": ["altitude"]}
functions.land_drone:
{"type": "object", "properties": {"location": {"type": "string", "enum": ["current", "home_base", "custom"]}, "coordinates": {"type": "object"}}, "required": ["location"]}
functions.control_drone_movement:
{"type": "object", "properties": {"direction": {"type": "string", "enum": ["forward", "backward", "left", "right", "up", "down"]}, "distance": {"type": "integer"}}, "required": ["direction", "distance"]}
functions.set_drone_speed:
{"type": "object", "properties": {"speed": {"type"

In [28]:
with open("eval_df.jsonl", "w") as f:
    eval_df.to_json(f, orient="records", lines=True)

In [29]:
eval_df_loaded = pd.read_json("eval_df.jsonl", orient="records", lines=True)
eval_df_loaded.head()

,inputs,ground_truth,predictions,similarity_metric
0,<|im_start|>system\nYou are an intelligent AI ...,"functions.takeoff_drone:\n{""altitude"": 100}</s>\n","Sure! To set the drone's altitude, you can use...",0.142494
1,<|im_start|>system\nYou are an intelligent AI ...,"functions.takeoff_drone:\n{""altitude"": 100}</s>\n","Sure, to set the drone's height, you can use t...",0.076280
2,<|im_start|>system\nYou are an intelligent AI ...,"functions.land_drone:\n{""location"": ""current""}...","Sure, I can bring the drone down to where you ...",0.047862
3,<|im_start|>system\nYou are an intelligent AI ...,"functions.land_drone:\n{""location"": ""current""}...","Sure, here's an updated version of the AI syst...",0.051860
4,<|im_start|>system\nYou are an intelligent AI ...,"functions.land_drone:\n{""location"": ""home_base...","To bring the drone back to base for landing, c...",0.066802
